In [1]:
import logging
import sys

In [2]:
root = logging.getLogger()
root.setLevel(logging.DEBUG)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.INFO)
formatter = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
handler.setFormatter(formatter)
root.addHandler(handler)

In [3]:
import os
os.environ["LITELLM_LOG"] = "DEBUG"
import litellm
litellm.set_verbose=True

02:01:51 - LiteLLM:DEBUG: utils.py:137 - Exception import enterprise features No module named 'litellm.proxy.enterprise'


In [4]:
# %load_ext autoreload
# %autoreload 2
# import sys; sys.path.append('/future/u/okhattab/repos/public/stanfordnlp/dspy')

import dspy
from dspy.evaluate import Evaluate
from dspy.datasets.gsm8k import GSM8K, gsm8k_metric
from dspy.teleprompt import BootstrapFewShotWithRandomSearch
from dspy.modeling import JSONBackend
from dspy.modeling import TextBackend
from dspy.modeling import ChatBackend

In [5]:
import phoenix as px

from openinference.semconv.resource import ResourceAttributes
from openinference.instrumentation.dspy import DSPyInstrumentor
# from clank.so.openinference.semconv.resource import ResourceAttributes
# from clank.so-openinference.instrumentation.dspy import DSPyInstrumentor
from opentelemetry import trace as trace_api
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk import trace as trace_sdk
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace.export import SimpleSpanProcessor
from openinference.semconv.trace import SpanAttributes

endpoint = "http://127.0.0.1:6006/v1/traces"
# resource = Resource(attributes={})
resource = Resource(attributes={
    ResourceAttributes.PROJECT_NAME: 'Span-test'
})
tracer_provider = trace_sdk.TracerProvider(resource=resource)
span_otlp_exporter = OTLPSpanExporter(endpoint=endpoint)
tracer_provider.add_span_processor(SimpleSpanProcessor(span_exporter=span_otlp_exporter))
trace_api.set_tracer_provider(tracer_provider=tracer_provider)
DSPyInstrumentor().instrument()

2024-06-25 02:01:55,110 - phoenix.config - INFO - 📋 Ensuring phoenix working directory: C:\Users\Felix\.phoenix
2024-06-25 02:01:55,128 - phoenix.inferences.inferences - INFO - Dataset: phoenix_dataset_1edf44dc-d252-4089-ba70-73f870a6874d initialized


In [6]:
gsm8k = GSM8K()


# backend = JSONBackend(model="ollama/codegemma:7b-code-q5_K_M", api_base="http://localhost:11435", params={"max_tokens": 500, "temperature": 0.3, "num_retries": 5, "repeat_penalty": 1.2, "top_p": 0.9})

#, "response_format": {"type": "json_object"}})

# backend = JSONBackend(model="ollama/llama3:70b", api_base="http://localhost:11434", params={"max_tokens": 500, "temperature": 0.3, "num_retries": 5, "repeat_penalty": 1.2, "top_p": 0.9})
# backend = JSONBackend(model="ollama/llama3:70b", api_base="http://localhost:11434", params={"max_tokens": 500, "temperature": 0.3, "num_retries": 5, "repeat_penalty": 1.2, "top_p": 0.9, "response_format": {"type": "json_object"}})


backend = TextBackend(model="ollama/llama3:70b", params={"max_tokens": 500, "temperature": 0.3, "num_retries": 5, "repeat_penalty": 1.2, "top_p": 0.9})
# backend = TextBackend(model="ollama/llama3:70b", params={"max_tokens": 500, "temperature": 0.3, "num_retries": 5, "repeat_penalty": 1.2, "top_p": 0.9, "response_format": {"type": "json_object"}})


# backend = ChatBackend(model="ollama/llama3:70b", api_base="http://localhost:11434", params={"max_tokens": 500, "temperature": 0.1, "num_retries": 5})

# backend = JSONBackend(
#     model="ollama/llama3:70b", 
#     params={
#         "max_tokens": 500, 
#         "temperature": 0.1, 
#         "num_retries": 5, 
#         "response_format": {
#             "type": "object",  # Added the "type" key here
#             "properties": {
#                 "question": {"title": "Question", "type": "string"},
#                 "rationale": {"title": "Rationale", "type": "string"},
#                 "answer": {"title": "Answer", "type": "string"}
#             }, 
#             "required": ["question", "rationale", "answer"]
#         }
#     }
# )


dspy.settings.configure(backend=backend)

trainset, devset = gsm8k.train[:10], gsm8k.dev[:10]

100%|██████████| 1319/1319 [00:00<00:00, 25084.73it/s]


In [7]:
# import dspy
# from dspy.datasets.gsm8k import GSM8K, gsm8k_metric

# # Set up the LM.
# # turbo = dspy.OpenAI(model="llama3:70b", api_key="sk-1234", api_base="http://localhost:4001/",model_type='chat')
# turbo = dspy.OpenAI(model='gpt-3.5-turbo', api_key="sk-1234", max_tokens=1000, api_base='http://localhost:4000', model_type="chat")
# dspy.settings.configure(lm=turbo)

# # Load math questions from the GSM8K dataset.
# gsm8k = GSM8K()
# gsm8k_trainset, gsm8k_devset = gsm8k.train[:10], gsm8k.dev[:10]

In [8]:
NUM_THREADS = 16
evaluate = Evaluate(devset=devset[:], metric=gsm8k_metric, num_threads=NUM_THREADS, display_progress=True, display_table=0)

In [9]:
class CoT(dspy.Module):
    def __init__(self):
        super().__init__()
        self.prog = dspy.ChainOfThought("question -> answer")
    
    def forward(self, question):
        return self.prog(question=question)

In [10]:
RUN_FROM_SCRATCH = True

if RUN_FROM_SCRATCH:
    config = dict(max_bootstrapped_demos=4, max_labeled_demos=4, num_threads=NUM_THREADS)
    teleprompter = BootstrapFewShotWithRandomSearch(metric=gsm8k_metric, **config)
    cot_bs = teleprompter.compile(CoT(), trainset=trainset, valset=devset)
    # cot_bs.save('turbo_8_8_10_gsm8k_200_300.json')
else:
    cot_bs = CoT()
    cot_bs.load('turbo_8_8_10_gsm8k_200_300.json')

2024-06-25 02:02:09,265 - dspy.teleprompt.random_search - INFO - 2024-06-25T06:02:09.265542Z [info     ] Going to sample between        [dspy.teleprompt.random_search] filename=random_search.py lineno=58 positional_args=(1, 'and', 4, 'traces per predictor.')
2024-06-25 02:02:09,267 - dspy.teleprompt.random_search - INFO - 2024-06-25T06:02:09.266542Z [info     ] Will attempt to train          [dspy.teleprompt.random_search] filename=random_search.py lineno=61 positional_args=(16, 'candidate sets.')


Average Metric: 10 / 10  (100.0): 100%|██████████| 10/10 [00:00<00:00, 114.70it/s]


2024-06-25 02:02:09,446 - dspy.evaluate.evaluate - INFO - 2024-06-25T06:02:09.446819Z [info     ] Average Metric: 10 / 10 (100.0%) [dspy.evaluate.evaluate] filename=evaluate.py lineno=200
2024-06-25 02:02:09,451 - dspy.teleprompt.random_search - INFO - 2024-06-25T06:02:09.451815Z [info     ] Score:                         [dspy.teleprompt.random_search] filename=random_search.py lineno=135 positional_args=(100.0, 'for set:', [0])
2024-06-25 02:02:09,452 - dspy.teleprompt.random_search - INFO - 2024-06-25T06:02:09.452818Z [info     ] New best sscore:               [dspy.teleprompt.random_search] filename=random_search.py lineno=138 positional_args=(100.0, 'for seed', -3)
2024-06-25 02:02:09,453 - dspy.teleprompt.random_search - INFO - 2024-06-25T06:02:09.453818Z [info     ] Scores so far: [100.0]         [dspy.teleprompt.random_search] filename=random_search.py lineno=142
2024-06-25 02:02:09,454 - dspy.teleprompt.random_search - INFO - 2024-06-25T06:02:09.454817Z [info     ] Best score:

  0%|          | 0/10 [00:00<?, ?it/s]

2024-06-25 02:02:09,648 - dspy.evaluate.evaluate - ERROR - 2024-06-25T06:02:09.648472Z [error    ] Error for example in dev set: 		 Example missing necessary output field: rationale [dspy.evaluate.evaluate] filename=evaluate.py lineno=180
2024-06-25 02:02:09,652 - dspy.evaluate.evaluate - ERROR - 2024-06-25T06:02:09.652320Z [error    ] Error for example in dev set: 		 Example missing necessary output field: rationale [dspy.evaluate.evaluate] filename=evaluate.py lineno=180


...

2024-06-25 02:02:09,665 - dspy.evaluate.evaluate - ERROR - 2024-06-25T06:02:09.665476Z [error    ] Error for example in dev set: 		 Example missing necessary output field: rationale [dspy.evaluate.evaluate] filename=evaluate.py lineno=180
2024-06-25 02:02:09,680 - dspy.evaluate.evaluate - ERROR - 2024-06-25T06:02:09.680922Z [error    ] Error for example in dev set: 		 Example missing necessary output field: rationale [dspy.evaluate.evaluate] filename=evaluate.py lineno=180
2024-06-25 02:02:09,690 - dspy.evaluate.evaluate - ERROR - 2024-06-25T06:02:09.690151Z [error    ] Error for example in dev set: 		 Example missing necessary output field: rationale [dspy.evaluate.evaluate] filename=evaluate.py lineno=180


Average Metric: 0.0 / 3  (0.0):  20%|██        | 2/10 [00:00<00:00, 44.02it/s]

2024-06-25 02:02:09,691 - dspy.evaluate.evaluate - ERROR - 2024-06-25T06:02:09.691226Z [error    ] Error for example in dev set: 		 Example missing necessary output field: rationale [dspy.evaluate.evaluate] filename=evaluate.py lineno=180


Average Metric: 0.0 / 6  (0.0):  50%|█████     | 5/10 [00:00<00:00, 89.10it/s]

Exception: Example missing necessary output field: rationale

In [ ]:
evaluate(cot_bs, devset=devset[:])

In [ ]:
print(backend.history[-1].prompt.to_str())